# SMC-based Constrained Decoding for InternLM2-Math-Plus

This notebook implements Sequential Monte Carlo (SMC) inference with constrained decoding for the InternLM2-Math-Plus model, specifically focusing on generating valid Lean code using the HFPPL library.

In [ ]:
# !git clone https://github.com/adamzweiger/hfppl
# %cd hfppl
%pip install .

Processing /Users/adamzweiger1/Desktop/Research/SMC/hfppl
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hfppl: filename=hfppl-0.1.0-py3-none-any.whl size=24182 sha256=148fa055b1c2a54f8b14be033f7ce6c4487912f732d90792d1f281b2fe439a9a
  Stored in directory: /private/var/folders/d2/1qht11gx2x1fhw8z_ttz0yb80000gn/T/pip-ephem-wheel-cache-82r0939y/wheels/28/f1/b0/0d9d729f137aa943d9fe8fb647ef663670bd8ba6092605ea49
Successfully built hfppl
  Attempting uninstall: hfppl
    Found existing installation: hfppl 0.1.0
    Uninstalling hfppl-0.1.0:
      Successfully uninstalled hfppl-0.1.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
%pip install einops


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from typing import List, Tuple, Optional, Dict
import re
from hfppl import Model, CachedCausalLM, Token, LMContext, smc_standard
from string import punctuation
from tqdm import tqdm
import json
from pathlib import Path
import asyncio
import os
import subprocess
import tempfile
import shutil
import time

In [4]:
# Install Lean and Lake
%cd lean_validator
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y
!~/.elan/bin/elan default leanprover/lean4:stable
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/.elan/bin")
result = subprocess.run(["lake", "--version"], capture_output=True, text=True)
print(result.stdout) # should be Lake version 5.0.0-7dc1ceb (Lean version 4.14.0-rc2)
!lake exe cache get
%cd ..

/Users/adamzweiger1/Desktop/Research/SMC/hfppl/lean_validator


/Users/adamzweiger1/Library/Caches/pypoetry/virtualenvs/hfppl-OVGc_7Le-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


info: downloading installer
info: updating existing elan installation

info: using existing install for 'leanprover/lean4:stable'
info: default toolchain set to 'leanprover/lean4:stable'

  leanprover/lean4:stable unchanged - Lean (version 4.13.0, arm64-apple-darwin23.6.0, commit 6d22e0e5cc5a, Release)

Lake version 5.0.0-7dc1ceb (Lean version 4.14.0-rc2)

No files to download
Decompressing 5602 file(s)
Unpacked in 251 ms
Completed successfully!
/Users/adamzweiger1/Desktop/Research/SMC/hfppl


/Users/adamzweiger1/Library/Caches/pypoetry/virtualenvs/hfppl-OVGc_7Le-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
def validate_lean_proof(theorem_string: str) -> bool:
    """
    Validates a Lean theorem string by integrating it into the LeanValidator project
    and attempting to build the project.

    Args:
        theorem_string (str): The Lean theorem to validate.

    Returns:
        bool: True if the theorem is valid (build succeeds), False otherwise.
    """
    current_dir = os.getcwd()

    # Define paths
    lean_project_dir = os.path.join(current_dir, 'lean_validator')
    lean_validator_dir = os.path.join(lean_project_dir, 'LeanValidator')
    lean_validator_lean = os.path.join(lean_project_dir, 'LeanValidator.lean')

    # Path for the temporary theorem file
    temp_file_name = 'Temp.lean'
    temp_file_path = os.path.join(lean_validator_dir, temp_file_name)

    # Backup path
    lean_validator_backup = lean_validator_lean + '.bak'

    try:
        # Backup the original LeanValidator.lean
        shutil.copyfile(lean_validator_lean, lean_validator_backup)

        # Create the temporary theorem file with necessary imports
        with open(temp_file_path, 'w') as tmp_file:
            tmp_file.write('import Mathlib\n\n')
            tmp_file.write(theorem_string)

        # Append import statement to LeanValidator.lean
        with open(lean_validator_lean, 'a') as lv_file:
            lv_file.write('\nimport LeanValidator.Temp\n')

        # Run 'lake build' in the lean_validator project directory
        result = subprocess.run(['lake', 'build'], cwd=lean_project_dir, capture_output=True, text=True)

        # Return build success status
        print("\nTheorem_string: " + theorem_string + "\nResult: " + str(result.returncode==0) + "\n" + "-"*50)
        return result.returncode == 0

    except Exception as e:
        print(f"An error occurred: {e}")
        return False

    finally:
        # Restore the original LeanValidator.lean
        if os.path.exists(lean_validator_backup):
            shutil.move(lean_validator_backup, lean_validator_lean)

        # Remove the temporary theorem file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [6]:
def validate_lean_line(theorem_string: str, proof_lines: list[str], candidate_line: str) -> str:
    """
    Validates a line of Lean code within the context of an existing theorem proof.

    Args:
        theorem_string (str): The full theorem statement (including 'theorem ... :=')
        proof_lines (list[str]): List of proof lines that come before the candidate line
        candidate_line (str): The line to validate in this context

    Returns:
        str: "finished" if the line completes the proof,
             "valid" if the line is valid but needs more steps,
             "invalid" if the line is not valid in this context
    """
    temp_file_path = None
    try:
        # Set up the Lean project directory (adjust the path as needed)
        lean_project_dir = os.path.join(os.getcwd(), 'lean_validator')

        # Ensure the Lean project directory exists
        if not os.path.isdir(lean_project_dir):
            raise FileNotFoundError(f"Lean project directory '{lean_project_dir}' does not exist.")

        # Create a temporary Lean file in the project directory
        with tempfile.NamedTemporaryFile('w', delete=False, suffix='.lean', dir=lean_project_dir) as tmp_file:
            temp_file_path = tmp_file.name
            # Write the Lean code to the temporary file
            tmp_file.write('import Mathlib\n\n')
            tmp_file.write(theorem_string + '\n')
            for line in proof_lines:
                tmp_file.write('  ' + line.strip() + '\n')
            tmp_file.write('  ' + candidate_line.strip() + '\n')

        # Run Lean once using Lake's environment
        result = subprocess.run(
            ['lake', 'env', 'lean', temp_file_path],
            cwd=lean_project_dir,
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            # No errors: Proof is finished
            return "finished"

        # Analyze the error output to determine the type of failure
        std_output = result.stdout.lower()
        # Extract lines containing 'error:'
        error_lines = [line for line in std_output.splitlines() if 'error:' in line]

        if not error_lines:
            # If there are errors but none contain 'error:', treat as invalid
            return "invalid"

        # Check if all error lines are about 'unsolved goals'
        all_unsolved = True
        for line in error_lines:
            if 'unsolved goals' not in line:
                all_unsolved = False
                break

        if all_unsolved:
            return "valid"
        else:
            return "invalid"

    except Exception as e:
        print(f"An error occurred: {e}")
        return "invalid"

    finally:
        # Clean up: Remove the temporary file
        if temp_file_path and os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [7]:
# Example usage with different scenarios
theorem = "theorem example_add_positive (a b : ℕ) (ha : 0 < a) (hb : 0 < b) : 0 < a + b := by"

# Test 1: Valid intermediate step
print("Test 1: Valid intermediate step")
result = validate_lean_line(theorem, [], "have : 0 + 0 < a + b := add_lt_add ha hb")
print(f"Result: {result}\n")

# Test 2: Complete proof
print("Test 2: Complete proof")
previous_lines = ["have : 0 + 0 < a + b := add_lt_add ha hb"]
result = validate_lean_line(theorem, previous_lines, "exact this")
print(f"Result: {result}\n")

# Test 3: Invalid step
print("Test 3: Invalid step")
result = validate_lean_line(theorem, [], "exact this")
print(f"Result: {result}")

Test 1: Valid intermediate step
Result: valid

Test 2: Complete proof
Result: finished

Test 3: Invalid step
Result: invalid


In [8]:
# Initialize model
model = CachedCausalLM.from_pretrained("internlm/internlm2-math-plus-1_8b") # 1.8 B model
# model = CachedCausalLM.from_pretrained("internlm/internlm2-math-plus-7b")
model.batch_size = 16

In [9]:
class LeanProofModel(Model):
    def __init__(self, lm: CachedCausalLM, theorem_content: str, max_tokens: int = 100):
        super().__init__()
        self.lm = lm
        self.theorem_content = theorem_content.strip()
        prompt = f"{self.theorem_content}\n"
        self.context = LMContext(lm, prompt)
        self.max_tokens = max_tokens
        self.n_tokens = 0
        self.current_line = ''
        self.proof_lines = []
        self.finished = False

    async def start(self):
        # Initialize the context with the theorem statement
        self.n_tokens = 0

    async def step(self):
        # Sample next token
        token = await self.sample(self.context.next_token())
        self.n_tokens += 1

        # Get the token text
        token_id = token.token_id
        token_text = self.lm.tokenizer.decode([token_id], clean_up_tokenization_spaces=False)

        # Update the current line
        self.current_line += token_text

        # Check if a line has finished (newline character)
        if '\n' in token_text or self.n_tokens >= self.max_tokens:
            # Split the current line at the newline character
            lines = self.current_line.split('\n')
            # The first part is the completed line
            completed_line = lines[0].strip()
            # The rest (if any) is the beginning of the next line
            self.current_line = '\n'.join(lines[1:]) if len(lines) > 1 else ''

            if completed_line:
                # Add the completed line to proof_lines
                self.proof_lines.append(completed_line)

                # Validate the completed line
                validation_result = validate_lean_line(self.theorem_content, self.proof_lines[:-1], completed_line)

                if validation_result == "finished":
                    self.condition(True)
                    self.finish()
                    return
                elif validation_result == "invalid":
                    self.condition(False)
                    self.finish()
                    return
                # If valid, continue sampling
                else:
                    pass

        # If maximum tokens reached without finishing
        if self.n_tokens >= self.max_tokens:
            self.condition(False)
            self.finish()
            return

        # Optional: Print the current context for debugging
        print(str(self.context) + "\n" + "-"*50 + "\n")

    def immutable_properties(self):
        return {"theorem_content", "max_tokens"}

In [10]:
async def smc_inference(model, theorem_content: str, max_new_tokens: int = 200, n_particles: int = 16):
    lean_model = LeanProofModel(model, theorem_content, max_tokens=max_new_tokens)
    particles = await smc_standard(
        model=lean_model,
        n_particles=n_particles,
        ess_threshold=0.5
        # visualization_dir="results"
    )
    return [str(p.context) for p in particles]

In [11]:
# theorem_content = """theorem amc12a_2015_p10 (x y : ℤ) (h₀ : 0 < y) (h₁ : y < x) (h₂ : x + y + (x * y) = 80) : x = 26 := by"""
theorem_content = """theorem example_add_positive (a b : ℕ) (ha : 0 < a) (hb : 0 < b) : 0 < a + b := by"""

overall_start_time = time.perf_counter()

proofs = await smc_inference(model, theorem_content, max_new_tokens=200, n_particles=16)

overall_end_time = time.perf_counter()
overall_duration = overall_end_time - overall_start_time

print(f"Total time taken for evaluation: {overall_duration:.2f} seconds")

 --
--------------------------------------------------

 push
--------------------------------------------------

  
--------------------------------------------------

 lin
--------------------------------------------------

 r
--------------------------------------------------

  
--------------------------------------------------

  
--------------------------------------------------

 --
--------------------------------------------------

 --
--------------------------------------------------

 --
--------------------------------------------------

 rw
--------------------------------------------------

 rw
--------------------------------------------------

  
--------------------------------------------------

 cd
--------------------------------------------------

 --
--------------------------------------------------

 --
--------------------------------------------------

 -- assume
--------------------------------------------------

 push_cast
--------------------------------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 push_cast

--------------------------------------------------

  linarith
--------------------------------------------------



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 rwa [
--------------------------------------------------

  cases hb
--------------------------------------------------

  apply Nat
--------------------------------------------------

 -- when using
--------------------------------------------------

 -- We apply
--------------------------------------------------

 -- we prove
--------------------------------------------------

 rw [N
--------------------------------------------------

 rw [add
--------------------------------------------------

  apply Nat
--------------------------------------------------

 cdiazm
--------------------------------------------------

 -- first prove
--------------------------------------------------

 -- (除
--------------------------------------------------

 push_cast
 lin
--------------------------------------------------



...

  cases hb
  case inl hb => exact Nat.lt_add_right
--------------------------------------------------



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  cases hb
  case inl hb => exact Nat.lt_add_right b
--------------------------------------------------

  cases hb
  case inl hb => exact Nat.lt_add_right b a
--------------------------------------------------

  cases hb
  case inl hb => exact Nat.lt_add_right b a ha
--------------------------------------------------

  cases hb
  case inl hb => exact Nat.lt_add_right b a ha hb
--------------------------------------------------



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Total time taken for evaluation: 128.00 seconds


In [12]:
for idx, proof in enumerate(proofs):
    print(f"Proof {idx+1}:\n{proof}\n{'='*50}\n")

Proof 1:
 rw [Nat.add_comm]
 apply Nats.add_pos' ha hb



Proof 2:
 linarith


Proof 3:
 -- when using the theorem, you don't need to prove the statement explicitly


Proof 4:
 linarith


Proof 5:
  cases hb
  case inl hb => exact Nat.lt_add_right b a ha hb


Proof 6:
 -- when using the theorem, you don't need to prove the statement manually


Proof 7:
 -- when using the theorem, you don't need to prove the statement


Proof 8:
 push_cast
 linarith


Proof 9:
 linarith


Proof 10:
 rw [Nat.add_comm]
 apply Nats.add_pos' ha hb


Proof 11:
 linarith


Proof 12:
 linarith


Proof 13:
 rw [Nat.add_comm]
 apply Nats.add_pos' ha hb


Proof 14:
 push_cast
 linarith


Proof 15:
 linarith


Proof 16:
 rw [Nat.add_comm]
 apply Nats.add_pos' ha hb


